In [2]:
import networkx as nx
from rdkit import Chem
import pandas
import numpy as np

In [3]:

def smile_to_graph2(smile, normalize= True): #normally False

 
    ## use RDKit library to read in the smile string
    ## using this library allows us to visualize the mol. w/ respect to the chemical properties of it
    mol = Chem.MolFromSmiles(smile)

    ## number of atoms in the 
    c_size = mol.GetNumAtoms()

   
    # empty feature vector for each atom
    features = []

    # create a graph (from NetworkX library)
    g = nx.Graph()

 
    
    for atom in mol.GetAtoms():

        ### add the feature vector at the atom ####

        feature = atom_features(atom)

        if normalize:      

            features.append(feature/sum(feature))

        else:

            ## append features to the array
            features.append(feature) 

        
        # GetIDX: Returns the atom’s index (ordering in the molecule)
        atom_idx = atom.GetIdx()              

        g.add_node(atom_idx)                  

    

    # edges = []

    # g = nx.Graph()

    
    ## GetBonds ==> Returns a read-only sequence of the atom’s bonds (tuple)
    
    ## add edges for the nodes ##
    for bond in mol.GetBonds():

        #edges.append([bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()])

        # GetBeginAtomIdx  ==> index of bond's first atom
        g.add_edge(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx())

   

    nodes_del = []

 

    for node in g:

        # degree = number of edges adjacent to the node
        # so if the node/atom doesn't have any bonds, we delete it ? why?
        if g.degree(node) == 0:

           nodes_del.append(node)

 

   

           

    for indx in nodes_del:    

       g.remove_node(indx)    

 
    # filter out the atoms we don't want to include
    F = [element for index, element in enumerate(features) if index not in nodes_del]           

    

    # g = nx.Graph(edges).to_directed()

    # g = graph
    # F = filtered feature vector
    # c_size = num of atoms
    # mol = molecule we're working with
    # return g, F, c_size, mol
    return F
 

In [4]:
def atom_features(atom):

    return np.array([atom.GetDegree(),

                     atom.GetTotalNumHs(),

                     atom.GetImplicitValence(),

                     int(atom.GetIsAromatic()),

                     #atom.GetAtomicNum(),

                     #atom.GetTotalValence(),

                     #atom.GetTotalDegree(),

                     #atom.GetFormalCharge(),

                     #atom.GetPropsAsDict().get("_GasteigerCharge", 0),

                     #atom.GetPropsAsDict().get("_TriposAtomType", 0),

                     #atom.GetNumRadicalElectrons(),

                     #int(atom.IsInRing())

                     ])


In [5]:
from tdc.single_pred import ADME
data = ADME(name = 'hia_hou')
data.get_data()

Found local copy...
Loading...
Done!


,Drug_ID,Drug,Y
0,Acetanilide,CC(=O)Nc1ccccc1,1
1,Acetazolamide.mol,CC(=O)Nc1nnc(S(N)(=O)=O)s1,1
2,Alfacalcidol,C=C1/C(=C\C=C2/CCC[C@]3(C)[C@@H]2CC[C@H]3[C@@H...,1
3,Aminopyrine,Cc1c(N(C)C)c(=N)n(-c2ccccc2)n1C,1
4,Amosulalol.mol,COc1ccccc1OCCNC[C@@H](O)c1ccc(C)cc1S(N)(=O)=O,1
...,...,...,...
573,Tiludronic_acid,O=P(O)(O)C(Cc1ccc(Cl)cc1)P(=O)(O)O,0
574,Zanamivir.mol,CC(=O)N[C@H]1[C@@H]([C@@H](O)[C@H](O)CO)OC(C(=...,0
575,Kanamycin.mol,CC(=O)NC[C@@H]1O[C@@H](O[C@H]2[C@@H](N)C[C@H](...,0
576,Amikacin.mol,NCC[C@@H](O)C(=O)N[C@@H]1C[C@H](N)[C@H](O[C@@H...,0


In [44]:
df = data.get_data()
num_rows = df.shape[0]

In [45]:
num_rows

578

In [58]:
row_index_1 = 1  # The row index you want to access
row_index_2 = 2
column_name = 'Drug'  # The column name you want to access

def calculate_emd_distance(df, row_index_1, row_index_2, column_name):
    # Select drug molecules from the df
    drug1 = df.loc[row_index_1, column_name]
    drug2 = df.loc[row_index_2, column_name]

    # SMILE string to graph
    x = smile_to_graph2(drug1)
    y = smile_to_graph2(drug2)

    x_array = np.array(x)
    y_array = np.array(y)

    # Loss matrix
    M = ot.dist(x_array, y_array, metric="sqeuclidean")

    # Number of points in each distribution
    n_x = x_array.shape[0]
    n_y = y_array.shape[0]

    # Create uniform weight vectors
    weights_x = np.ones(n_x) / n_x
    weights_y = np.ones(n_y) / n_y

    # Calculate the EMD using the squared Euclidean distance matrix
    emd_distance = ot.emd2(weights_x, weights_y, M)

    return emd_distance

# Sanity check: Let's calculate EMD for two drugs
calculate_emd_distance(df, row_index_1, row_index_2, column_name)

0.21698316461863257

In [75]:
num_rows = 3
print(num_rows)

3


In [88]:
# Pariwise comparision of all the drugs in the dataframe

# Initialize an empty matrix to store EMD distances

# sample of first 4 drugs
num_rows = 4 # change 4 --> df.shape[0]
emd_matrix = np.zeros((num_rows, num_rows))

# Calculate EMD distances and fill the matrix
for i in range(3): #change 3 --> num_rows
    for j in range(i + 1, num_rows):
        emd_distance = calculate_emd_distance(df, i, j, column_name)
        emd_matrix[i, j] = emd_distance
        emd_matrix[j, i] = emd_distance  # Symmetric matrix

# Now, emd_matrix contains the EMD distances between all pairs of rows

# Create a similarity matrix (inverse of EMD distances)
similarity_matrix = 1 / (1 + emd_matrix)

In [89]:
# # Display the similarity matrix (test: first 4 drugs)
print("Similarity Matrix:")
print(similarity_matrix)

Similarity Matrix:
[[1.         0.88742435 0.93507143 0.93395946]
 [0.88742435 1.         0.82170405 0.8784956 ]
 [0.93507143 0.82170405 1.         0.92512607]
 [0.93395946 0.8784956  0.92512607 1.        ]]


In [87]:
print(similarity_matrix.shape)

(4, 4)


In [ ]:
#to do: look into kernals, support vector regression, support vector machine, applications of 
# similarity matrix